In [13]:
library(limma)
library(DESeq2)
library(edgeR)
library(dcov)

In [14]:
###################################
## Here we do the normalizations ##
###################################

## Read and format raw RNA-seq data to matrix 
## We remove duplicated gene names (ad-hoc!)
rna.counts <- read.table("../Data/CCLE_RNAseq_genes_counts_20180929.gct", header = TRUE, row.names=1, sep = "\t", skip = 2)
rna.counts$Description <- NULL
rna.counts <- as.matrix(rna.counts)


In [15]:
## Total counts normalization
rna.norm <- t(t(rna.counts)/rowSums(t(rna.counts)))
rna.norm.tc <- rna.norm*mean(rowSums(t(rna.counts)))
rna.norm.ltc <- log2(rna.norm.tc + 1)
write.csv(rna.norm.ltc, file = "../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.logTC.normalized.csv", quote=F)

In [16]:
## Quantile normalization
## (first log transform an then use limma)
rna.counts.log2 <- log2(rna.counts + 1) 
rna.norm.quant <- normalizeQuantiles(rna.counts.log2, ties=TRUE)
write.csv(rna.norm.quant, file = "../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.logQUANT.normalized.csv", quote=F)



In [17]:
## Convert to SummarizedExperiment and create DESeq2 object
## We use tissue type for col data (note you can set all the same)
tissue <- substring(colnames(rna.counts), regexpr("_", colnames(rna.counts)) + 1, nchar(colnames(rna.counts)))
col.data <- data.frame(TISSUE = tissue)
col.data$TISSUE <- factor(col.data$TISSUE)
rna.dss <- DESeqDataSetFromMatrix(countData = rna.counts, colData = col.data, design = ~1) # for DEG, set design = ~TISSUE 
rna.dss <- DESeq(rna.dss)


converting counts to integer mode

Warning message in DESeq(rna.dss):
“the design is ~ 1 (just an intercept). is this intended?”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 14335 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [18]:
## We have several ways of normalizing after creating the DESeq2 object
## see here: http://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#count-data-transformations
rna.dss.vst <- vst(rna.dss, blind=FALSE)
rna.norm.vst <- assay(rna.dss.vst)
write.csv(rna.norm.vst, file = "../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.VST.normalized.csv", quote=F)


In [19]:
## What they call TMM normalization is explained here:
## https://www.biostars.org/p/9475236/ and https://gtpb.github.io/ADER18F/pages/tutorial1.html
y <- DGEList(counts=rna.counts, genes=rownames(rna.counts), group = tissue)
y <- calcNormFactors(y)
cpms.log <- cpm(y, log=TRUE)
write.csv(cpms.log, file = "../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.logCPMS.normalized.csv", quote=F)


In [2]:
## Finally, we do the log transform of RPKM
rna.rpkm <- read.table("../Data/CCLE_RNAseq_genes_rpkm_20180929.gct", header = TRUE, row.names=1,skip = 2, sep = "\t")
rna.rpkm[,1] <- NULL
rna.norm.lrpkm <- log2(rna.rpkm + 1.0)
write.csv(rna.norm.lrpkm, file = "../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.logRPKM.normalized.csv", quote=F)


In [21]:
## We can also inspect DGE analysis results with DESeq2 object
## set design = ~TISSUE above (will take very long time to run!)
#rna.res <- results(rna.dss, contrast=c("TISSUE", "KIDNEY", "BREAST"))
#res.ordered <- as.data.frame(rna.res[order(rna.res$pvalue),])

In [23]:
met <- read.csv("../Data/original_metabolite_data.csv", row.names = 1)
met[,1] <- NULL
met <- t(met)

rna.norm.logTC <- read.csv("../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.logTC.normalized.csv", row.names = 1)
rna.norm.logRPKM <- read.csv("../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.logRPKM.normalized.csv", row.names = 1)
rna.norm.logCPMS <- read.csv("../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.logCPMS.normalized.csv", row.names = 1)
rna.norm.logDESeq2 <- read.csv("../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.logDESeq2.normalized.csv", row.names = 1)
rna.norm.logQUANT <- read.csv("../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.logQUANT.normalized.csv", row.names = 1)
rna.norm.VST <- read.csv("../Data/Generated Data/CCLE_RNAseq_genes_counts_20180929.VST.normalized.csv", row.names = 1)

cells <- intersect(colnames(met), colnames(rna.norm.logTC))
met <- met[,cells]
rna.norm.logTC <- rna.norm.logTC[,cells]
rna.norm.logRPKM <- rna.norm.logRPKM[,cells]
rna.norm.logCPMS <- rna.norm.logCPMS[,cells]
rna.norm.logDESeq2 <- rna.norm.logDESeq2[,cells]
rna.norm.logQUANT <- rna.norm.logQUANT[,cells]
rna.norm.VST <- rna.norm.VST[,cells]

## Perform distance correlations
res.logTC <- dcor.test(t(met),t(rna.norm.logTC), R = 500)
res.logRPKM <- dcor.test(t(met),t(rna.norm.logRPKM), R = 500)
res.logCPMS <- dcor.test(t(met),t(rna.norm.logCPMS), R = 500)
res.logDESeq2 <- dcor.test(t(met),t(rna.norm.logDESeq2), R = 500)
res.logQUANT <- dcor.test(t(met),t(rna.norm.logQUANT), R = 500)
res.VST <- dcor.test(t(met),t(rna.norm.VST), R = 500)

res.logTC$statistic/mean(res.logTC$replicates)
res.logRPKM$statistic/mean(res.logRPKM$replicates)
res.logCPMS$statistic/mean(res.logCPMS$replicates)
res.logDESeq2$statistic/mean(res.logDESeq2$replicates)
res.logQUANT$statistic/mean(res.logQUANT$replicates)
res.VST$statistic/mean(res.VST$replicates)


[1] 4.254164

[1] 5.337135

[1] 4.57432

[1] 4.333556

[1] 4.306545

[1] 4.788288

In [38]:
res.logTC$p

[1] 0.001996008

In [33]:
res.VST$statistic

[1] 0.4611193

In [31]:
mean(res.VST$replicates)

[1] 0.09630151

In [28]:
rna.norm.VST

,DMS53_LUNG,SW1116_LARGE_INTESTINE,NCIH1694_LUNG,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,UMUC3_URINARY_TRACT,HOS_BONE,HUNS1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,RVH421_SKIN,⋯,NCIH2073_LUNG,MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HOP62_LUNG,EKVX_LUNG,UO31_KIDNEY,SF268_CENTRAL_NERVOUS_SYSTEM,SF539_CENTRAL_NERVOUS_SYSTEM,SNB75_CENTRAL_NERVOUS_SYSTEM,HOP92_LUNG,MUTZ3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000223972.4,4.639364,7.214159,3.825288,3.760011,3.723492,4.097507,3.145262,3.703737,4.558415,3.674347,⋯,5.514640,4.489464,3.613896,3.352404,4.273739,3.271716,3.775451,5.483657,4.014704,4.216938
ENSG00000227232.4,10.490867,10.250133,8.844650,9.025386,9.533250,9.874909,10.507708,9.450898,10.449060,9.297298,⋯,9.394643,10.532394,9.625658,9.971209,9.430172,9.638140,9.782959,9.156907,9.678653,10.409658
ENSG00000243485.2,3.360823,6.048727,3.471072,2.861416,2.890682,3.274206,3.145262,3.703737,3.738896,3.796614,⋯,4.152510,3.206840,2.894351,3.352404,4.473184,3.417823,3.105145,3.453169,4.014704,3.452368
ENSG00000237613.2,2.914350,6.413144,3.134320,2.861416,3.136699,3.274206,3.145262,3.306784,3.556176,3.796614,⋯,5.177624,3.753677,2.894351,3.352404,4.273739,3.095210,3.672748,3.453169,3.541748,4.471344
ENSG00000268020.2,2.281881,5.557979,2.888964,3.096138,2.281881,2.862240,2.281881,3.306784,3.033226,2.923759,⋯,4.057487,2.281881,3.141779,2.281881,3.741806,2.860747,3.431365,2.281881,3.381172,4.067570
ENSG00000240361.1,2.281881,6.180787,2.888964,3.272832,3.321553,2.862240,3.331829,2.881799,3.738896,3.376400,⋯,4.057487,2.281881,2.894351,3.352404,3.335263,3.758419,2.281881,3.453169,3.802959,4.216938
ENSG00000186092.4,3.169444,6.048727,3.318698,3.272832,2.890682,3.660064,3.485939,3.124396,4.157405,3.376400,⋯,4.323270,3.041310,3.327649,2.281881,4.160974,2.860747,3.672748,3.583125,3.541748,3.452368
ENSG00000238009.2,2.914350,5.797181,3.602775,3.096138,2.890682,3.547405,5.275057,3.587963,4.795012,4.007647,⋯,4.152510,3.463270,3.481207,4.440754,4.377306,2.860747,3.431365,3.302894,3.802959,8.543324
ENSG00000233750.3,5.470272,8.546095,5.140479,4.847416,5.182762,4.890823,7.340700,5.149997,6.045038,5.473295,⋯,7.039860,6.072325,5.005510,5.250483,6.136722,4.573447,5.026473,4.413596,5.417902,7.259712


In [29]:
met

,DMS53_LUNG,SW1116_LARGE_INTESTINE,NCIH1694_LUNG,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,UMUC3_URINARY_TRACT,HOS_BONE,HUNS1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,RVH421_SKIN,⋯,NCIH2073_LUNG,MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HOP62_LUNG,EKVX_LUNG,UO31_KIDNEY,SF268_CENTRAL_NERVOUS_SYSTEM,SF539_CENTRAL_NERVOUS_SYSTEM,SNB75_CENTRAL_NERVOUS_SYSTEM,HOP92_LUNG,MUTZ3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
X2.aminoadipate,6.112727,5.577413,5.886398,5.770030,5.480683,5.892011,6.234494,5.972748,6.041394,6.081218,⋯,5.884544,6.034226,5.923901,5.703092,6.095599,5.977636,5.957233,5.967707,5.962415,6.332344
X3.phosphoglycerate,6.034198,5.727045,5.574881,6.099229,5.469742,6.039330,6.072933,5.523737,5.474501,6.080584,⋯,5.904622,5.525035,6.067542,5.751252,6.052813,6.026483,6.090834,5.931487,5.992640,5.812531
alpha.glycerophosphate,5.896896,5.111468,5.541259,6.233259,6.509397,6.309562,6.123578,5.826709,6.767043,6.109159,⋯,5.701968,7.388613,5.969476,5.418494,5.894046,6.480536,5.323475,5.620542,6.296222,5.446330
X4.pyridoxate,6.000532,6.073250,5.848375,5.543495,6.251005,5.645860,5.509110,6.034410,6.265092,5.865165,⋯,6.301520,6.087225,6.011333,6.063868,6.122323,6.112056,6.145795,5.955261,5.916386,6.469052
aconitate,5.513618,5.802494,5.665026,5.767759,5.190578,5.501537,6.236506,6.097423,5.484676,6.064304,⋯,5.852829,5.340299,5.963644,5.742431,5.799900,5.878324,5.741201,5.935244,6.043984,5.501232
adenine,5.868529,5.824473,5.875548,6.155905,5.897085,5.975171,6.031743,6.133391,6.233674,5.782527,⋯,6.680147,6.237912,5.901777,6.031719,5.525050,5.732508,5.605435,5.384674,5.883976,6.469954
adipate,5.977177,5.888821,5.894904,6.111148,6.148333,5.907445,5.927760,6.196595,5.999579,5.614704,⋯,5.884313,5.938592,5.780635,5.875812,5.947329,5.981183,6.184192,5.955886,5.954731,6.136017
alpha.ketoglutarate,5.693074,5.768379,5.839640,5.949481,5.607481,5.180048,6.057326,6.092996,5.946140,6.203720,⋯,6.050884,5.647575,6.056975,5.702185,6.148747,6.459139,6.004195,6.239657,6.181054,5.479015
AMP,5.923737,5.760784,5.742613,6.342703,5.871600,7.005265,5.914833,7.451125,5.913747,6.008612,⋯,5.724812,7.038409,5.625687,5.641449,5.763445,5.642855,5.663866,5.527619,5.820966,5.694210
citrate,5.641242,5.914742,5.570208,6.054781,5.128463,5.435565,6.082849,6.078011,5.497771,6.078279,⋯,5.904078,5.239519,5.989435,5.986714,5.808916,6.039198,5.758910,5.973401,6.045409,5.078509


ERROR: Error in dcov.test(t(met), t(rna.norm.VST), R = 500): could not find function "dcov.test"
